# CrossFit scraped data cleanse

This notebook takes a step by step approach on how to cleanse raw data scraped from the previous notebook

In [2]:
import numpy as np
import pandas as pd
import re

import seaborn as sns

%matplotlib inline 
# to display charts in jupyter notebook

## 1. Import raw datascrape file

In [3]:
pd.set_option('display.max_columns', None)

In [5]:
df = pd.read_csv(r'C:\Users\suhaib.qazi\Desktop\cf_1a.csv', encoding = "ISO-8859-1")

#### 1.1 Drop itteration number - not used

In [6]:
df = df.drop(['Iter_Number'],axis=1)

In [7]:
df = df.reset_index(drop=True)

#### Titles include trailing spaces - so to make it easier to work with, get rid of these

In [8]:
df = df.rename(columns=lambda x: x.strip())

## 2. Number conversion

Convert all to metric

### 2.1 Convert Weightlifting figures to kg

For all weight fields (olympic lifting + athlete weight) standardise into kg. If weight is in pounds, convert (mutliply by 0.45)

In [9]:
for col in ['Back Squat','Clean and Jerk','Deadlift','Snatch','Weight']:
#1. is it lb or kg?
    df[col+' met']=df[col].str.strip().str[-2:] 
#2. Create multipliers to convert numbers. As we are converting to kg, if measure is in kg then x1, else x0.45    
    df[col+' met'] = pd.to_numeric(df[col+' met'].replace('kg',1).replace('lb',0.453592), errors='coerce')
#3. convert measures to numbers (getting rid of metric: "kg" or "lb")
    df[col] = pd.to_numeric(df[col].str.strip().str[:-2], errors='coerce')
#4. multiple weight number by multiplier    
    df[col]=df[col]*df[col+' met']
    df[col]=df[col].round()
#5. get rid of multiplier column    
    del df[col+' met']

### 2.2 Convert yearly Open position to number

e.g. 250th becomes 250

In [10]:
#for the year columns
for col in df.iloc[:,12:22]:
#convert to string to get substrings
        df[col] = df[col].apply(str)
#convert column to numeric - after deleting last 2 characters        
        df[col] = pd.to_numeric(df[col].str.strip().str[:-2], errors='coerce').round()

### 2.3 Convert Height

For Heights given in feet and inches (e.g. 5'9") convert to cm (e.g. 175cm)

In [11]:
#get rid of " (for inches) - this will allow us to split the inches into two list elements
df['Height'] = df['Height'].str.replace('"','')

function to convert - if list has two elements, in is ft/inches so convert to cm
if list is single item, get rid of 'cm'

In [12]:
def parse_height(x):
    feet_inches = x.split("'")
    if len(feet_inches) == 2:
        return 2.54 * (12 * int(feet_inches[0]) + float(feet_inches[1]))        
    else: return feet_inches[0][:-2] #list(map(int, results))
# Convert a height in inches to a height in centimeters

In [13]:
df['Height'] = pd.to_numeric(df['Height'].apply(parse_height))

In [14]:
#df['Height'] = pd.to_numeric(df['Height'], errors='coerce')

### 2.4 Convert Minute/Sec figures to seconds

In [15]:
def sec_converter(x):    
    min_sec_split = x.split(":")

    if len(min_sec_split) == 2:
        return int(min_sec_split[0])*60 + int(min_sec_split[1])
    else: return min_sec_split[0] 

In [16]:
for col in df:
    df[col] = df[col].replace('--', np.nan, regex=True)
    df[col] = df[col].replace(0, np.nan, regex=True)
    

In [17]:
for col in ['Filthy 50','Fran','Grace','Helen','Run 5k','Sprint 400m']:
    df[col] = df[col].apply(str)
    df[col] = df[col].apply(sec_converter)
    df[col] = pd.to_numeric(df[col], errors='coerce')

### 2.5 Extract gender information

Extract from Division titles

In [18]:
def gender(s):
    val = ''

    if any(word in str(s['Division']) for word in ['Women']):
        val = 'Female'
    elif any(word in str(s['Division']) for word in ['Men']):
        val = 'Male'
    else: val = None
    return val

In [19]:
df['gender']=df.apply(gender,axis=1)

In [20]:
df

,Back Squat,Clean and Jerk,Deadlift,Fight Gone Bad,Filthy 50,Fran,Grace,Helen,Max Pull-ups,Run 5k,Snatch,Sprint 400m,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,Affiliate,Age,Division,Height,Name,Region,Team,Weight,gender
0,140.0,90.0,190.0,322,1075.0,298.0,162.0,452.0,NaN,82.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35,NaN,180.00,Linus Mansson,NaN,NaN,79.0,None
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1591.0,1414.0,902.0,696.0,NaN,CrossFit Big D,47,Masters Men (45-49),167.64,Richard Neal,South Central,CrossFit Big D,66.0,Male
2,166.0,111.0,181.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CrossFit Ibirapuera,40,Masters Men (35-39),NaN,Leandro Leite,Latin America,NaN,91.0,Male
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,460.0,NaN,1114.0,NaN,NaN,CrossFit 253,51,Masters Men (50-54),12.70,Todd Burgess,North West,NaN,81.0,Male
4,134.0,102.0,211.0,412,1049.0,172.0,179.0,421.0,50,1201.0,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21848.0,15971.0,12094.0,NaN,CrossFit Manitowoc,39,Masters Men (35-39),182.88,Matt DiMarco,North Central,CrossFit Manitowoc,86.0,Male
5,107.0,82.0,NaN,NaN,NaN,NaN,166.0,NaN,NaN,NaN,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28678.0,NaN,CrossFit Oregon City,39,Masters Women (35-39),175.26,Jen Cereghino,North West,CFOC,65.0,Female
6,160.0,125.0,200.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,95.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21165.0,6909.0,4968.0,NaN,CrossFit Marilia,32,Individual Men,190.00,Robson Silva,Latin America,NaN,93.0,Male
7,NaN,51.0,119.0,269,NaN,NaN,NaN,NaN,NaN,1494.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31,NaN,167.64,Mackenzie Aubrey,NaN,NaN,69.0,None
8,120.0,NaN,138.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5011.0,NaN,Unaffiliated,36,Masters Women (35-39),162.56,Melyssa Gagnon,Canada East,NaN,52.0,Female
9,125.0,79.0,129.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,60,NaN,182.88,Jim McCarthy,NaN,NaN,102.0,None


## 3. Output cleansed data

In [ ]:
df.to_csv(r'C:\Users\suhaib.qazi\Desktop\CF_data_cleansed.csv', sep=',')